In [1]:
import os 

In [2]:
%pwd

'c:\\Users\\sakshi\\Documents\\projects\\disease_classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\sakshi\\Documents\\projects\\disease_classification'

In [11]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/sakshinayakc05/disease_classification.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="sakshinayakc05"
os.environ["MLFLOW_TRACKING_PASSWORD"]="42276643d608fc67150ceb61f008e50f1e905a2f"

In [7]:
import tensorflow as tf

In [8]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [10]:

from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [12]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/sakshinayakc05/disease_classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [13]:

import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [41]:
import os
import tempfile
import shutil

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            # Log parameters and metrics
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            
            # Log the scores.json file as an artifact
            if os.path.exists("scores.json"):
                mlflow.log_artifact("scores.json")
            
            # Try to log model with proper handling
            try:
                # Create a temporary directory for model saving
                temp_dir = tempfile.mkdtemp()
                model_path = os.path.join(temp_dir, "model.keras")
                
                # Save the model with .keras extension
                self.model.save(model_path, save_format='keras')
                
                # Log the entire model directory as an artifact
                mlflow.log_artifact(model_path, "model")
                
                print("Model logged successfully as artifact")
                
                # Clean up
                shutil.rmtree(temp_dir)
                
            except Exception as model_error:
                print(f"Model logging failed: {model_error}")
                print("Continuing without model logging...")
            
            print("Successfully logged experiment to MLflow")
            print(f"Loss: {self.score[0]:.4f}, Accuracy: {self.score[1]:.4f}")
            print(f"Tracking URI: {mlflow.get_tracking_uri()}")
            print(f"Experiment ID: {mlflow.active_run().info.experiment_id}")
            print(f"Run ID: {mlflow.active_run().info.run_id}")

In [42]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

Found 139 images belonging to 2 classes.
9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 672ms/step - accuracy: 0.9079 - loss: 0.1716


Model logged successfully as artifact
Successfully logged experiment to MLflow
Loss: 0.0808, Accuracy: 0.9640
Tracking URI: https://dagshub.com/sakshinayakc05/disease_classification.mlflow
Experiment ID: 0
Run ID: 1914a0b1c4d140aea9bd412975764925


In [27]:
import mlflow
print(mlflow.__version__)


2.2.2
